<a href="https://colab.research.google.com/github/jatoogunhyo/dart_disclosure/blob/main/SEC_edgar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [(링크) EDGARTOOLS 라이브러리 사용안내](https://github.com/dgunning/edgartools)

# 필요한 환경 설정

In [1]:
!pip install edgartools
from edgar import *
import re
import pandas as pd
import time
from tqdm.notebook import tqdm  # 구글 코랩에 적합한 tqdm 모듈
import requests

# SEC 서식
(배경) SEC에 제출해야 하는 공시는 다음과 같음

| 서식 유형                | 호출 코드 | 서식명                | 설명                                                                 |
|--------------------------|-----------|-----------------------|----------------------------------------------------------------------|
| **재무 보고서**          | 10-K      | 연례 보고서           | 기업의 연간 재무 상태와 운영 성과를 상세히 보고하는 문서입니다.         |
|                          | 10-Q      | 분기 보고서           | 기업의 분기별 재무 상태와 운영 성과를 보고하는 문서입니다.             |
|                          | 20-F      | Form 20-F            | 외국 기업이 SEC에 제출하는 연례 보고서입니다.                         |
|                          | 40-F      | Form 40-F            | 캐나다 기업이 SEC에 제출하는 연례 보고서입니다.                       |
| **중요 사건 보고서**      | 8-K       | 현재 보고서           | 기업의 중요한 사건이나 변경 사항을 즉시 보고하는 문서입니다.          |
| **소유권 및 투자 보고서** | SC 13D    | Schedule 13D         | 5% 이상 주식을 보유한 투자자가 제출하는 문서로, 보유 목적과 계획을 상세히 기술합니다. |
|                          | SC 13G    | Schedule 13G         | 5% 이상 주식을 보유한 수동적 투자자가 제출하는 간소화된 보고서입니다. |
|                          | 13F-HR    | Form 13F             | 기관투자자가 보유한 주식 목록을 분기별로 보고하는 문서입니다.         |
| **내부자 거래 보고서**    | 3         | Form 3               | 기업의 임원, 이사, 또는 주요 주주가 된 개인이 자신의 소유권을 처음으로 보고하는 문서입니다. |
|                          | 4         | Form 4               | 기업 내부자의 주식 거래를 보고하는 문서입니다.                       |
|                          | 5         | Form 5               | 보고되지 않은 내부자 거래를 연간으로 보고하는 문서입니다.             |
| **증권 등록**            | S-1       | Form S-1             | 기업이 IPO를 통해 주식을 공개적으로 발행할 때 사용하는 등록 문서입니다. |
|                          | S-3       | Form S-3             | 이미 상장된 기업이 추가 증권을 발행할 때 사용하는 간소화된 등록 문서입니다. |
|                          | S-4       | Form S-4             | 기업 합병이나 인수 시 사용하는 등록 문서입니다.                       |
|                          | S-8       | Form S-8             | 직원 스톡 옵션 계획 등을 위해 증권을 등록할 때 사용하는 문서입니다.   |
|                          | F-1       | Form F-1             | 외국 기업이 미국에서 증권을 발행할 때 사용하는 등록 문서입니다.       |
|                          | F-3       | Form F-3             | 외국 상장 기업이 추가 증권을 발행할 때 사용하는 간소화된 등록 문서입니다. |
|                          | F-4       | Form F-4             | 외국 기업의 합병이나 인수 시 사용하는 등록 문서입니다.               |
|                          | F-6       | Form F-6             | 미국 주식 예탁 증서(ADR)를 등록할 때 사용하는 문서입니다.             |
| **기타 등록 및 공시**     | D         | Form D               | 증권의 면제를 신청할 때 사용하는 문서로, 주로 사모펀드에서 사용됩니다. |
|                          | 144       | Form 144             | 제한된 증권의 매각을 계획할 때 사용하는 통지서입니다.                 |
|                          | EFFECT    | Notice of Effectiveness | 증권 등록이 SEC에 의해 효력을 얻었음을 알리는 통지서입니다.        |
| **크라우드펀딩 보고서**   | C         | Form C               | 크라우드펀딩을 통해 자금을 모집하는 기업이 사용하는 문서입니다.       |
|                          | C-U       | Form C-U             | 크라우드펀딩 진행 상황을 업데이트하는 문서입니다.                     |
|                          | C-AR      | Form C-AR            | 크라우드펀딩 완료 후 연례 보고서입니다.                              |
|                          | C-TR      | Form C-TR            | 크라우드펀딩 종료 보고서입니다.                                      |
| **포트폴리오 보고서**     | NPORT-P   | Form NPORT-P         | 투자 회사의 월간 포트폴리오 보고서입니다.                           |
| **기타**                 | MA-I      | Form MA-I            | Municipal Advisor의 초기 등록 시 사용하는 문서입니다.               |


# 서식별 문서 확인하기

## 확인할 공시서식 지정하기
* form 파라미터에 지정하면 됨

In [2]:
# SEC API를 사용하려면 본인의 이메일을 전송해야함
set_identity("JaeKwang jaekwang.jung@krx.co.kr")

# 서식명, 연도를 사용해서 서식 불러오기
filings = get_filings(form=['8-K'], year=2024)
filings.to_pandas()

,form,company,cik,filing_date,accession_number
0,8-K,180 Life Sciences Corp.,1690080,2024-12-31,0001213900-24-113675
1,8-K,1847 Holdings LLC,1599407,2024-12-31,0001213900-24-114023
2,8-K,"ALLURION TECHNOLOGIES, INC.",1964979,2024-12-31,0001193125-24-287091
3,8-K,"APPLIED OPTOELECTRONICS, INC.",1158114,2024-12-31,0001683168-24-009068
4,8-K,ASPAC III Acquisition Corp.,1890361,2024-12-31,0001213900-24-113961
...,...,...,...,...,...
70024,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000007
70025,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000009
70026,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000011
70027,8-K/A,PISMO COAST VILLAGE INC,216877,2024-01-02,0001513162-24-000013


In [3]:
# 하나를 예시로 보기
filing = filings[16]
filing

# 공시를 보고싶으면 Primary Document에 대당하는 URL을 클릭하면 됨

╭───────────────────────────────── BERKSHIRE HATHAWAY ENERGY CO [1081316] 8-K 📄 ─────────────────────────────────╮
│ ╭──────────────────────┬────────────╮                                                                           │
│ │ 0001081316-24-000029 │ 2024-12-31 │                                                                           │
│ ╰──────────────────────┴────────────╯                                                                           │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮                  │
│ │ Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                             │                  │
│ ├────────────────────────────────────────────────────────────────────────────────────────────┤                  │
│ │ 🏠 https://sec.gov/Archives/edgar/data/1081316/0001081316-24-000029-index.html             │                  │
│ │ 📄 https://sec.gov/Archives/edgar/data/1081316/000108131624000029/bhe-2024

# 주요사항보고서(Current Report, 8-K) Item별 내용 확인하기

## 8-K 보고대상 항목(item)
(배경) 8-K 보고대상 항목은 다음과 같음


| **Item**       | **Official Name**                                       | **한국어 명칭**                        | **한국어 설명**                                                                                 |
|----------------|-------------------------------------------------------|---------------------------------------|---------------------------------------------------------------------------------------------|
| Item 1.01      | Entry into a Material Definitive Agreement             | 중요 확정 계약 체결                    | 회사가 중요한 확정 계약을 체결했을 때 이를 공시합니다.                                         |
| Item 1.02      | Termination of a Material Definitive Agreement         | 중요 확정 계약 해지                    | 회사가 체결한 중요 확정 계약이 종료되었을 때 이를 공시합니다.                                   |
| Item 1.03      | Bankruptcy or Receivership                             | 파산 또는 법정관리                     | 회사가 파산 절차를 시작하거나 법정관리에 들어갔을 때 이를 공시합니다.                           |
| Item 2.01      | Completion of Acquisition or Disposition of Assets     | 자산 취득 또는 처분 완료               | 중요한 자산의 인수나 처분이 완료되었을 때 이를 공시합니다.                                     |
| Item 2.02      | Results of Operations and Financial Condition          | 영업 실적 및 재무 상태                  | 회사의 영업 실적이나 재무 상태와 관련된 중요한 업데이트를 공시합니다.                           |
| Item 2.03      | Creation of a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement of a Registrant | 직접적인 재무 의무 발생 또는 비상각형 약정                     | 새로운 재무적 의무가 발생했을 때 이를 공시합니다.                                               |
| Item 2.04      | Triggering Events That Accelerate or Increase a Direct Financial Obligation or an Obligation under an Off-Balance Sheet Arrangement | 재무 의무 가속 또는 증가를 초래하는 사건 | 특정 사건으로 인해 재무 의무가 증가하거나 가속화되었을 때 이를 공시합니다.                       |
| Item 2.05      | Costs Associated with Exit or Disposal Activities      | 사업 철수 또는 폐기에 따른 비용         | 회사가 사업을 철수하거나 자산을 폐기하는 활동에 따른 비용을 공시합니다.                         |
| Item 2.06      | Material Impairments                                   | 중요한 자산 감액                       | 회사가 중요한 자산의 감액을 인식했을 때 이를 공시합니다.                                        |
| Item 3.01      | Notice of Delisting or Failure to Satisfy a Continued Listing Rule or Standard; Transfer of Listing | 상장 폐지 통보 또는 상장 요건 미충족 통보 | 상장 폐지 통보를 받거나 상장 요건을 충족하지 못했을 때 이를 공시합니다.                         |
| Item 3.02      | Unregistered Sales of Equity Securities                | 비등록 주식 판매                       | 비등록 주식을 판매했을 때 이를 공시합니다.                                                     |
| Item 3.03      | Material Modification to Rights of Security Holders    | 증권 보유자의 권리에 대한 중요한 수정 | 증권 보유자의 권리 변경과 관련된 중요한 사항을 공시합니다.                                       |
| Item 4.01      | Changes in Registrant’s Certifying Accountant          | 회계법인 변경                          | 회사의 회계법인 변경과 관련된 사항을 공시합니다.                                                |
| Item 4.02      | Non-Reliance on Previously Issued Financial Statements or a Related Audit Report or Completed Interim Review | 기존 재무제표 또는 감사 보고서 불신 | 기존 재무제표나 감사 보고서를 더 이상 신뢰할 수 없게 되었을 때 이를 공시합니다.                 |
| Item 5.01      | Changes in Control of Registrant                       | 지배권 변경                            | 회사의 지배권 변경이 발생했을 때 이를 공시합니다.                                               |
| Item 5.02      | Departure of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers; Compensatory Arrangements of Certain Officers | 이사 또는 특정 임원의 퇴임, 선출, 임명 | 이사 및 특정 임원의 퇴임, 선출, 임명, 보상 계약 관련 사항을 공시합니다.                          |
| Item 5.03      | Amendments to Articles of Incorporation or Bylaws; Change in Fiscal Year | 정관 또는 내규 수정, 회계연도 변경   | 정관 또는 내규 수정이나 회계연도 변경이 발생했을 때 이를 공시합니다.                             |
| Item 5.04      | Temporary Suspension of Trading Under Registrant’s Employee Benefit Plans | 직원 복리후생 계획 관련 거래 중단     | 직원 복리후생 계획과 관련된 거래 중단을 공시합니다.                                              |
| Item 5.05      | Amendments to the Registrant’s Code of Ethics, or Waiver of a Provision of the Code of Ethics | 윤리강령 수정 또는 면제                 | 윤리강령의 수정 또는 특정 조항의 면제를 공시합니다.                                             |
| Item 6.01      | ABS Informational and Computational Material           | ABS 정보 및 계산 자료                  | 자산유동화증권(ABS) 관련 정보와 계산 자료를 공시합니다.                                          |
| Item 6.02      | Change of Servicer or Trustee                          | 서비스 제공자 또는 신탁인 변경          | 자산유동화증권의 서비스 제공자나 신탁인이 변경되었을 때 이를 공시합니다.                         |
| Item 6.03      | Change in Credit Enhancement or Other External Support | 신용 보강 또는 외부 지원 변경          | 신용 보강 방법이나 기타 외부 지원의 변경을 공시합니다.                                           |
| Item 6.04      | Failure to Make a Required Distribution                | 필수 분배금 미지급                     | 자산유동화증권과 관련된 필수 분배금을 지급하지 못했을 때 이를 공시합니다.                        |
| Item 6.05      | Securities Act Updating Disclosure                     | 증권법 갱신 공시                       | 증권법과 관련된 갱신 공시를 제공합니다.                                                        |
| Item 7.01      | Regulation FD Disclosure                               | 공정공시 규정 공시                     | 공정공시 규정에 따른 공시 사항을 제공합니다.                                                   |
| Item 8.01      | Other Events                                           | 기타 사건                              | 위의 항목에 해당하지 않는 기타 중요한 사건을 공시합니다.                                        |
| Item 9.01      | Financial Statements and Exhibits                      | 재무제표 및 부속문서                    | 제출된 재무제표 및 관련 문서를 공시합니다.                                                      |


## 8-K 공시 불러오기

❗**주의** ❗
전체 공시를 대상으로 하면 부하가 걸림
전수조사 목적으로 코드를 실행시엔 사전에 협의할 것

🙂 이번 코드에서는 나스닥 상장사 상위 100개사가 제출한 8-K 공시를 대상으로 수행

* 나스닥 종목코드 가져오기

In [4]:
# SEC에서 제공하는 CIK 목록
url_ciks = 'https://www.sec.gov/files/company_tickers_exchange.json'
# headers는 SEC에서 이메일 입력을 요구함
headers = {'User-Agent' : 'JaeKwang (jaekwang.jung@krx.co.kr)'}
# 불러오기
res = requests.get(url_ciks, headers=headers)
cik_list = res.json()
cik_df = pd.DataFrame(cik_list['data'], columns=cik_list['fields'])

# 나스닥 상장종목으로 저장
# 정렬 순서는 시가총액 순으로 '추정'됨
nasdaq = cik_df[cik_df['exchange'] == 'Nasdaq']

# 나스닥 상장 티커 목록
nasdaq_tickers = nasdaq['ticker'].tolist()

# 수집대상을 상위로만 한정
tickers = nasdaq_tickers[:100]

* 공시 불러와서 필터하기

In [5]:
# 나스닥 상위 100개사가 2024년에 제출한 8-K서식에 대해 불러오기
# Edgartools 라이브러리에서는 일단 서식, 연도에 대해서는 다 불러와야 함
filings = get_filings(form=['8-K'], year=2024)
# 불러온 공시항목에서 티커를 적용하여 필터
filings = filings.filter(ticker = tickers)
filings.to_pandas()

,form,company,cik,filing_date,accession_number
0,8-K,XCEL ENERGY INC,72903,2024-12-31,0000072903-24-000209
1,8-K,MICROSTRATEGY Inc,1050446,2024-12-30,0001193125-24-286217
2,8-K,FASTENAL CO,815556,2024-12-27,0000815556-24-000049
3,8-K,"Datadog, Inc.",1561550,2024-12-23,0001561550-24-000240
4,8-K,MICROSTRATEGY Inc,1050446,2024-12-23,0001193125-24-283686
...,...,...,...,...,...
986,8-K,Baker Hughes Co,1701605,2024-01-05,0001193125-24-003365
987,8-K,INTEL CORP,50863,2024-01-03,0000050863-24-000003
988,8-K,"Interactive Brokers Group, Inc.",1381197,2024-01-02,0001381197-24-000003
989,8-K,O REILLY AUTOMOTIVE INC,898173,2024-01-02,0000898173-24-000003


## 8-K 보고대상 Item을 식별하는 함수 만들기
* edgartools 라이브러리에서는 item을 식별해주는 함수는 없음
* 그러나 아래의 함수는 있으므로, 두 가지를 조합하여 함수 생성
  - 보고서 내용을 섹션化하여 돌려주는 함수
  - 정규표현식과 일치하는 섹션만 돌려주는 함수

In [6]:
def extract_sections_to_dataframe(filing):
    # 절차 1: filing.search() 실행(정규표현식에 해당하는 섹션을 찾는 과정)
    search_results = filing.search(r'(Item\s*|\xa0*\d+\.\d+).*?(?=Item\s*|\xa0*\d+\.\d+|$)', regex=True)

    # 절차 2: "Item"으로 시작하는 섹션의 인덱스 추출
    index_list = [
        result.loc for result in search_results
        if re.match(r'^Item', result.doc.strip(), re.IGNORECASE)
    ]

    # 절차 3: filing.sections() 함수를 사용하여 전체 섹션 가져오기
    all_sections = filing.sections()

    # 절차 4: 전체 섹션 중 item title과 item 본문을 발라내기
    # 제목, item_no, 그리고 내용을 저장할 리스트
    item_nos = []
    titles = []
    contents = []

    # 인덱스 리스트를 사용해 제목과 내용 분리
    for i, index in enumerate(index_list):
        # 현재 섹션 텍스트 가져오기
        section_text = all_sections[index].strip()

        # 제목을 item_no와 title로 분리
        match = re.match(r'^(Item\s*\d+\.\d+)\s*(.*)', section_text, re.IGNORECASE)
        if match:
            item_no = match.group(1)  # Item 번호
            title = match.group(2).strip()  # 제목의 나머지 부분
        else:
            item_no = "Unknown"
            title = section_text

        # 내용: 현재 인덱스부터 다음 인덱스 직전까지 섹션 합치기
        if i < len(index_list) - 1:  # 마지막이 아닌 경우
            content_sections = [all_sections[j].strip() for j in range(index + 1, index_list[i + 1])]

            if content_sections:  # 내용이 존재하면 합치기
                content = "\n".join(content_sections)
            else:  # 내용이 없으면 현재 섹션을 내용으로 사용
                content = section_text
        else:  # 마지막 인덱스인 경우 끝까지 합치기
            content_sections = [all_sections[j].strip() for j in range(index + 1, len(all_sections))]

            if content_sections:  # 내용이 존재하면 합치기
                content = "\n".join(content_sections)
            else:  # 내용이 없으면 현재 섹션을 내용으로 사용
                content = section_text

        # 리스트에 추가
        item_nos.append(item_no)
        titles.append(title)
        contents.append(content)

    # 데이터프레임 생성
    df = pd.DataFrame({"item_no": item_nos, "title": titles, "content": contents})

    return df

## Item 식별함수를 사용하여 데이터 수집하기

In [7]:
# 결과 저장 리스트
dataframes = []

# 기존에 받아온 filings를 데이터프레임으로 저장
# 공시기업, 일자 등 정보를 받아오기 위함임
df_filings = filings.to_pandas()

# Loop을 돌면서 DataFrame 생성
for i in tqdm(range(10), desc="Processing filings"):
    # df_filings 데이터프레임으로부터 이번 loop의 공시 정보 추출
    form = df_filings.loc[i, "form"]
    company = df_filings.loc[i, "company"]
    cik = df_filings.loc[i, "cik"]
    filing_date = df_filings.loc[i, "filing_date"]
    accession_number = df_filings.loc[i, "accession_number"]
    url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{accession_number}-index.htm'

    # 이번 loop의 filing 객체 추출
    filing = filings[i]
    # 앞서 만든 함수를 사용하여 item을 추출
    df = extract_sections_to_dataframe(filing)
    # 이번 loop의 공시정보를 칼럼값으로 추가하기
    df['order'] = str(i)
    df['form'] = form
    df['company'] = company
    df['cik'] = cik
    df['filing_date'] = filing_date
    df['accession_number'] = accession_number
    df['url'] = url

    # 칼럼순서 정리하기
    df = df[['order', 'form', 'company', 'cik', 'filing_date', 'accession_number', 'item_no', 'title', 'content', 'url']]

    # 데이터프레임 추가하기
    dataframes.append(df)

    # IP 차단 방지를 위한 쉬어가기
    time.sleep(0.2)

# 모든 DataFrame을 하나로 합치기
final_df = pd.concat(dataframes, ignore_index=True)

Processing filings:   0%|          | 0/10 [00:00<?, ?it/s]

In [8]:
# 확인해보기
final_df

,order,form,company,cik,filing_date,accession_number,item_no,title,content,url
0,0,8-K,XCEL ENERGY INC,72903,2024-12-31,0000072903-24-000209,Item 8.01,. Other Events,The SRP includes the following measures:\n•Dis...,https://www.sec.gov/Archives/edgar/data/72903/...
1,1,8-K,MICROSTRATEGY Inc,1050446,2024-12-30,0001193125-24-286217,Item 8.01,. Other Events.,"ATM Update \n As previously disclosed, on Octo...",https://www.sec.gov/Archives/edgar/data/105044...
2,1,8-K,MICROSTRATEGY Inc,1050446,2024-12-30,0001193125-24-286217,Item 7.01,Regulation FD Disclosure.,"BTC Yield KPI \n From October 1, 2024 to Decem...",https://www.sec.gov/Archives/edgar/data/105044...
3,2,8-K,FASTENAL CO,815556,2024-12-27,0000815556-24-000049,Item 5.02,(b). Departure of Directors or Certain Officer...,Item 5.02(b). Departure of Directors or Certai...,https://www.sec.gov/Archives/edgar/data/815556...
4,2,8-K,FASTENAL CO,815556,2024-12-27,0000815556-24-000049,Item 9.01,. Financial Statements and Exhibits.,Exhibit Description of Document ...,https://www.sec.gov/Archives/edgar/data/815556...
5,3,8-K,"Datadog, Inc.",1561550,2024-12-23,0001561550-24-000240,Item 5.02,Departure of Directors or Certain Officers; El...,"Datadog, Inc. ...",https://www.sec.gov/Archives/edgar/data/156155...
6,4,8-K,MICROSTRATEGY Inc,1050446,2024-12-23,0001193125-24-283686,Item 8.01,. Other Events.,"ATM Update \n As previously disclosed, on Octo...",https://www.sec.gov/Archives/edgar/data/105044...
7,4,8-K,MICROSTRATEGY Inc,1050446,2024-12-23,0001193125-24-283686,Item 7.01,Regulation FD Disclosure.,"BTC Yield KPI \n From October 1, 2024 to Decem...",https://www.sec.gov/Archives/edgar/data/105044...
8,5,8-K,ARCH CAPITAL GROUP LTD.,947484,2024-12-20,0000947484-24-000154,ITEM 8.01,Other Events.,ARCH CAPITAL GROUP LTD. ...,https://www.sec.gov/Archives/edgar/data/947484...
9,6,8-K,MICROSTRATEGY Inc,1050446,2024-12-20,0001193125-24-283170,Item 5.02,. Departure of Directors or Certain Officers...,"On December 20, 2024, the Board of Directors (...",https://www.sec.gov/Archives/edgar/data/105044...


## 수집한 내용 로컬에 엑셀로 다운받기

In [37]:
from google.colab import files

# 엑셀 파일로 저장
final_df.to_excel('nasdaq_upper100_8k_filings.xlsx', index=False)

# 파일 다운로드
files.download('nasdaq_upper100_8k_filings.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>